In [2]:
#coding=utf8
import scipy.io as scio
import numpy as np

In [3]:
# 以下是mat转numpy用的，训练时不需要运行，在此注掉了
# 'ktr_X','ktr_Y','kte_X','kte_Y','val_X','val_Y','dte_X','dte_Y'

datas = scio.loadmat('../data/nina_db1/Fopgs2_multiset_20220611T161411.mat')

print(datas['ktr_X'].shape)
Xtrain = np.transpose(datas['ktr_X'],[3,2,0,1])
Xtest = np.transpose(datas['kte_X'],[3,2,0,1])
Xval = np.transpose(datas['val_X'],[3,2,0,1])
Xdtest = np.transpose(datas['dte_X'],[3,2,0,1])
print(Xtrain.shape,'\n',Xtest.shape,'\n',\
    Xval.shape,'\n',Xdtest.shape)
# 标签
# print(datas1[0,0,:,:])
Ytrain = datas['ktr_Y']
Ytest = datas['kte_Y']
Yval = datas['val_Y']
Ydtest = datas['dte_Y']
print(Ytrain.shape,'\n',Ytest.shape,'\n',\
    Yval.shape,'\n',Ydtest.shape)
print(Ytrain.shape,'\n',len(Ytrain),Ytrain[1,0])
# print(len(datalb))
# print(datas1[0,0,:,:])

# print(datas['Y_TrainP'].ndim)
# datalb = datas['Y_TrainP']
# print(len(datalb))
# print(datalb[1,0])
# emg1 = datalb[1,0]
# type(emg1)
# emg1.astype(np.float32)
# np.save('../data/trainY.npy',datalb)
# print(datalb[1,0])
dataset = {}
dataset['Ytrain'] = Ytrain
dataset['Xtrain'] = Xtrain
dataset['Xtest'] = Xtest
dataset['Ytest'] = Ytest
dataset['Xval'] = Xval
dataset['Yval'] = Yval
dataset['Xdtest'] = Xdtest
dataset['Ydtest'] = Ydtest
np.save('../data/OpenganDataSet2213.npy',dataset)

In [3]:
# 把用于挑选 GAN 的验证集标签重标注，重新标注应该包括所有数据，即ForpyopenGAN20220103T155713.mat中所有样本都需要，方便后续使用：
# 在验证集中，所有前10类数据标注1，42类标注0，代表已知未知类。训练集不必重标注，测试集后续估计有用，这里亦按照此标准重标注
# 'tr_X','tr_Y','te_X','te_Y','gte_X','gte_Y'

# datas = scio.loadmat('../data/ForpyopenGAN20220103T155713.mat')
datas = scio.loadmat('../data/SignalAcquisitionForpyopenGAN20220223T224326.mat')
print(datas['tr_X'].shape)
Xtrain = np.transpose(datas['tr_X'],[3,2,0,1])
Xtest = np.transpose(datas['te_X'],[3,2,0,1])
Xval = np.transpose(datas['gte_X'],[3,2,0,1])
# Xdtest = np.transpose(datas['dte_X'],[3,2,0,1])
print(Xtrain.shape,'\n',Xtest.shape,'\n',\
    Xval.shape,'\n')
# 标签
# print(datas1[0,0,:,:])
Ytrain = datas['tr_Y']
Ytest = datas['te_Y']
Yval = datas['gte_Y']
# Ydtest = datas['dte_Y']
print(Ytrain.shape,'\n',Ytest.shape,'\n',\
    Yval.shape,'\n')
print(Ytrain.shape,'\n',len(Ytrain),Ytrain[1,0])
# print(len(datalb))
# print(datas1[0,0,:,:])

# print(datas['Y_TrainP'].ndim)
# datalb = datas['Y_TrainP']
# print(len(datalb))
# print(datalb[1,0])
# emg1 = datalb[1,0]
# type(emg1)
# emg1.astype(np.float32)
# np.save('../data/trainY.npy',datalb)
# print(datalb[1,0])
dataset = {}
dataset['Ytrain'] = Ytrain
dataset['Xtrain'] = Xtrain
dataset['Xtest'] = Xtest
dataset['Ytest'] = Ytest
dataset['Xval'] = Xval
dataset['Yval'] = Yval
# dataset['Xdtest'] = Xdtest
# dataset['Ydtest'] = Ydtest
np.save('../data/OpenganDataSet_220611a_0611.npy',dataset)

(200, 6, 1, 8362)
(8362, 1, 200, 6) 
 (3673, 1, 200, 6) 
 (6220, 1, 200, 6) 

(8362, 1) 
 (3673, 1) 
 (6220, 1) 

(8362, 1) 
 8362 0


In [4]:
#coding=utf8
import torch
import torch.nn as nn
# import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
import hiddenlayer as h
from visdom import Visdom
import datetime
import os
from utils.reuse import *
from utils.networks import *
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 自定义数据集类

transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
])
# 自定义神经网络,CNN
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32,\
             kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32,\
             kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, \
            kernel_size=3, padding=0)

        self.fc1 = nn.Linear(in_features=32 * 18 * 8, out_features=128)
        self.out = nn.Linear(in_features=128, out_features=10)
        self.dr1 = nn.Dropout2d(0.2)

    def forward(self, t):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=1)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        # t = self.dr1(t)
        t = F.max_pool2d(t, kernel_size=2, stride=1)

        # (4) hidden linear layer
        t = t.reshape(-1, 32 * 18 * 8)
        t = self.fc1(t)
        t = F.relu(t)
        t = self.dr1(t)

        # (5) output layer
        t = self.out(t)

        return t
class EMGDataset(Dataset):
 
    def __init__(self, data, label):
        self.data = data
        self.label = label
        self.transforms = transforms.ToTensor()
 
    def __getitem__(self, index):
        emgData = self.data[index,:,:,:]
        emgData = np.squeeze(emgData)#似乎不应该压缩了
        emglabel = self.label[index]
        emglabel = emglabel.astype(np.int16)
        emgData = self.transforms(emgData)      
        
        return emgData,emglabel
 
    def __len__(self):
        return len(self.label)
 
# dataarray = np.load('../data/OpenganDataSet2213.npy',allow_pickle=True)

# CNNdataset = dataset.item()
# print(type(dataset))
# traindata = dataset['Xtrain']
# trainlabel = dataset['Ytrain']
# testdata = dataset['Xtest']
# testlabel = dataset['Ytest']
# trainlabel = trainlabel[:,0]
# testlabel = testlabel[:,0]
dataset['Ytrain'] = Ytrain
dataset['Xtrain'] = Xtrain
dataset['Xtest'] = Xtest
dataset['Ytest'] = Ytest
dataset['Xval'] = Xval
dataset['Yval'] = Yval
dataset['Xdtest'] = Xdtest
dataset['Ydtest'] = Ydtest

<class 'dict'>


In [5]:

train_set = EMGDataset(Xtrain, Ytrain)
test_set = EMGDataset(Xtest, Ytest)
test_set_g = EMGDataset(Xdtest,Ydtest)
val_set = EMGDataset(Xval,Yval)

# 计算数据集的网络预测输出并构建新的特征向量四维 array
# 注意迭代器只能建立一次，然后以此前进，所以这里单列一行，避免重复建设
# 达成类似目标的做法有很多种，例如 append() 等，这里先完成功能，优化待后来同学了

def emgdata_to_net_preds(data_set,net_vector):
    batchl = iter(data_set)
    emg_vec = [torch.tensor([],requires_grad=False) for i in range(len(data_set.label))]
    for idx, _ in enumerate(emg_vec):
        sample_data,sample_label = next(batchl)
        sample_data = sample_data.to(torch.float32).unsqueeze(0)
        sample_label = torch.as_tensor(sample_label).long()
        emg_vec[idx] = net_vector(sample_data).detach().numpy()
    emg_vec_np = np.array(emg_vec)
    emg_vec_np = emg_vec_np[:,np.newaxis,:,:]
    return emg_vec_np
# 使用训练好的模型，用其输出构建新数据集
# 这段感觉应该放在数据处理的，但好像不是那么好切割开，就先这样吧
checkpoint_for_vector = torch.load('./c10s2022_01_03_20_17_36.pth')
net_vector = Network()
net_vector.load_state_dict(checkpoint_for_vector)
net_vector.eval()
emg_vec_npp_tr = emgdata_to_net_preds(data_set=train_set, net_vector = net_vector)
emg_vec_npp_te = emgdata_to_net_preds(data_set=test_set, net_vector = net_vector)
emg_vec_npp_teg = emgdata_to_net_preds(data_set=test_set_g, net_vector = net_vector)
emg_vec_npp_val = emgdata_to_net_preds(data_set=val_set, net_vector = net_vector)



In [8]:
def emgdata_label_tsp_01(dataset):
    emg_vec = []
    batch = iter(dataset)
    for _, label in batch:
        if label<10:
            emg_vec.append(1)
        else:
            emg_vec.append(0)
    return emg_vec

emg_vec_val_lb = emgdata_label_tsp_01(val_set)
emg_vec_dte_lb = emgdata_label_tsp_01(test_set_g)
emg_vec_te_lb = emgdata_label_tsp_01(test_set)


In [9]:
opg_fea_full = {}
opg_fea_full['Ytrain'] = Ytrain
opg_fea_full['Xtrain'] = emg_vec_npp_tr
opg_fea_full['Xtest'] = emg_vec_npp_te
opg_fea_full['Ytest'] = Ytest
opg_fea_full['Xval'] = emg_vec_npp_val
opg_fea_full['Yval'] = Yval
opg_fea_full['Xdtest'] = emg_vec_npp_teg
opg_fea_full['Ydtest'] = Ydtest

opg_fea_full['Yd_nlb'] = emg_vec_dte_lb
opg_fea_full['val_nlb'] = emg_vec_val_lb
opg_fea_full['Yte_nlb'] = emg_vec_te_lb
np.save('../data/OpenganDatafea_full_2222.npy',opg_fea_full)    

In [ ]:
# 也许会用到的一些备用代码
# # EMG需要转化成6维特征，用到刚刚训练的模型
# checkpoint_net_fe = torch.load('.//ckp//opg_0217a//oth//c_ep_200_acc9763_2022_02_26_23_18_44.pth')
# net_fe = Network_CNN_6ch_6cls_smr_v1()
# net_fe.load_state_dict(checkpoint_net_fe['model'])
# net_fe.eval()
# emg_vec_npp_tr = emgdata_to_net_preds(data_set=train_set, net_vector = net_fe)
# emg_vec_npp_te = emgdata_to_net_preds(data_set=test_set, net_vector = net_fe)
# emg_vec_npp_val = emgdata_to_net_preds(data_set=val_set, net_vector = net_fe)
# # 特征重组为新的数据集
# opg_fea_full = {}
# opg_fea_full['Ytrain'] = CNNdataset['Ytrain']
# opg_fea_full['Xtrain'] = emg_vec_npp_tr
# opg_fea_full['Xtest'] = emg_vec_npp_te
# opg_fea_full['Ytest'] = CNNdataset['Ytest']
# opg_fea_full['Xval'] = emg_vec_npp_val
# opg_fea_full['Yval'] = CNNdataset['Yval']
# np.save('../data/OpenganDatafea_smr_10cl_220227.npy',opg_fea_full)  

In [ ]:
# 也许会用到的一些备用代码
# 1Ytrain-20210721T193346.mat

# datas = scio.loadmat('../data/Y_TrainP.mat')
# print(datas['Y_TrainP'].ndim)
# datalb = datas['Y_TrainP']
# print(len(datalb))
# print(datalb[1,0])
# emg1 = datalb[1,0]
# type(emg1)
# emg1.astype(np.float32)
# np.save('../data/trainY.npy',datalb)
# print(datalb[1,0])

In [ ]:
# 这里检查了数据形状与维度，训练时不需要，注掉了

# tx = np.load('../data/trainX.npy')
# txp = tx[1,:,:,:]
# print(txp.shape)
# print(np.squeeze(txp).shape)

220521,组建包含所有需要数据（原始样本，CNN特征输出，手势动作标签，已知未知类标签）的大数据集

In [11]:
# 220521,组建包含所有需要数据（原始样本，CNN特征输出，手势动作标签，已知未知类标签）的大数据集
# 加载原始数据集
# datas = scio.loadmat("../data/SignalAcqForpyopenGAN20220520T165247.mat")

datas = scio.loadmat("../data/SignalAcqForpyopenGAN20220522T091046.mat")
# 后缀 known 是只含已知类的数据集，opset 表示既有已知也有未知，这里测试集known是验证集opset的子集，取了前六个已知类
Xtrain_known = np.transpose(datas["tr_X"], [3, 2, 0, 1])
Xtest_known = np.transpose(datas["te_X"], [3, 2, 0, 1])
Xval_opset = np.transpose(datas["gte_X"], [3, 2, 0, 1])
Ytrain_known = datas["tr_Y"]
Ytest_known = datas["te_Y"]
Yval_opset = datas["gte_Y"]
# mtv 表示真实动作标签，上面的opset 只是指示开闭的01值
Yval_opset_mtv = datas["gte_YT"]

In [12]:
# 加载CNN输出特征集，这个集只是将 X train，test,val 过了 CNN 之后变成特征值，所以这里实际上也只需要这些
GAN_dataarray = np.load("../data/OpenganDatafea_smr_10cl_220227.npy", allow_pickle=True)
GANdataset = GAN_dataarray.item()
print(type(GANdataset))
X_train_Net_output = GANdataset["Xtrain"]
# trainlabel = GANdataset["Ytrain"]
X_test_Net_output = GANdataset['Xtest']
# testlabel = GANdataset['Ytest']
X_val_Net_output = GANdataset["Xval"]
# vallabel = GANdataset["Yval"]


<class 'dict'>


In [13]:
# 现在组合数据集
opg_fea_full = {}
opg_fea_full["Xtrain_known"] = Xtrain_known
opg_fea_full["Xtest_known"] = Xtest_known
opg_fea_full["Xval_opset"] = Xval_opset
opg_fea_full["Ytrain_known"] = Ytrain_known
opg_fea_full["Ytest_known"] = Ytest_known
opg_fea_full["Yval_opset"] = Yval_opset

opg_fea_full["X_train_Net_output"] = X_train_Net_output
opg_fea_full["X_test_Net_output"] = X_test_Net_output
opg_fea_full["X_val_Net_output"] = X_val_Net_output

opg_fea_full["Yval_opset_mtv"] = Yval_opset_mtv
np.save("../data/OpgData_full_smr_10cl_220522_2.npy", opg_fea_full)